In [ ]:
!rm -rf /content/*
%pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=1VIKqcHtcBpQlwSKB3h7Q_Z-kDVlr21IC

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('district-wise-rainfall-data-for-india-2014.csv',sep=';')
df.columns = ['INDIAN_STATES_NAME', 'DISTRICTS_NAME', 'DATE', 'FREQUENCY','VARIABLE_NAME', 'VALUE', 'VARIABLE_NOTES']
df['DATE'] = pd.to_datetime(df['DATE'])
df.head()

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df['INDIAN_STATES_NAME'].value_counts()[:5]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15,3))
df['INDIAN_STATES_NAME'].value_counts()[:5].plot(kind="bar")
plt.xticks(rotation=90)
plt.show()

In [ ]:
df['INDIAN_STATES_NAME'].value_counts()

In [ ]:
df[df['INDIAN_STATES_NAME']=='Maharashtra'].DISTRICTS_NAME.value_counts()

In [ ]:
data = df[df['DISTRICTS_NAME'] == 'Dhule'].sort_values(by='DATE')
data.head()

In [ ]:
data.tail()

In [ ]:
s = '% Dep. are the Departures of rainfall from the long period averages of rainfall for the District.'
data = data[data.VARIABLE_NOTES != s]
data.head()

In [ ]:
df.columns

In [ ]:
#tmp.columns

In [ ]:
# tmp = df.groupby("INDIAN_STATES_NAME")["VALUE"].sum().reset_index().sort_values(by=['VALUE'],ascending=False)
# tmp.head()

# plt.figure(figsize=(15,3))
# df.plot(x=['INDIAN_STATES_NAME'], y=["VALUE"], kind="bar")
# plt.xticks(rotation=90)
# plt.show()

In [ ]:
data = data[['DATE','VALUE']]
data.index= data['DATE']
data = data.drop(columns=['DATE'])
data.head()

In [ ]:
data.tail()

In [ ]:
idx = pd.date_range(min(data.index), max(data.index),freq='MS')
data = data.reindex(idx, fill_value=np.nan)
data.head()

In [ ]:
data.tail()

In [ ]:
data.isna().sum()

In [ ]:
data = data.reset_index()
data=data.rename(columns={'index':'ds', 'VALUE':'y'})
data.head()

In [ ]:
%pip install pystan==2.19.1.1 prophet

In [ ]:
from prophet import Prophet
model = Prophet()
model.fit(data)

In [ ]:
future = model.make_future_dataframe(periods=24, freq = 'm')
future.tail()

In [ ]:
forecast = model.predict(future)
forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast.tail()

In [ ]:
model.plot(forecast);

In [ ]:
future_date = pd.DataFrame({'ds':['2022-02-01']})
forecast = model.predict(future_date)
forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast.tail()

In [ ]:
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_idx = list(range(1,13))
month = dict(zip(month_name,month_idx))

In [ ]:
month

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='720 days', period='180 days', horizon = '365 days')
df_cv.tail()

In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
df_p.head()

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mae')